## Einführung in Pandas

Ein kleines Tutorial in einige der wichtigsten Funktionen von pandas um Datensätze zu anaylsieren.

In [ ]:
import pandas as pd

Wir lesen die einzelnen CSV Dateien als pandas dataframes ein.

In [ ]:
pop = pd.read_csv('../data/external/GermanSongLyricsCorpus/Pop.csv')
rap = pd.read_csv('../data/external/GermanSongLyricsCorpus/Rap.csv')
rock = pd.read_csv('../data/external/GermanSongLyricsCorpus/Rock.csv')
schlager = pd.read_csv('../data/external/GermanSongLyricsCorpus/Schlager.csv')

Mit `.head()` rufen wir die ersten (5) Einträge eines Dataframes auf.

In [ ]:
pop.head()

Wir führen die vier Datensätze zu einem zusammen.

In [ ]:
df = pd.concat([pop, rap, rock, schlager], ignore_index=True)

Mit `dtypes` können wir uns die Typen der Spalten des Datensatzes anschauen.


In [ ]:
df.dtypes

"Genre" wurde in dem Datensatz falsch geschrieben. Also bennen wir die Spalte um.

In [ ]:
df.rename(columns={'gerne': 'genre'}, inplace=True)
df.head()

Wir sehen, dass bei `lyrics` immer `NaN` (not a number) steht. Die Songtexte sind in diesem Datensatz nicht vorhanden. Wir können diese Spalte aus dem Datensatz löschen.

In [ ]:
df.drop(['lyrics'], axis=1, inplace=True)
df.head()

Wieviele Einträge gibt es insgesamt?

In [ ]:
len(df)

Wieviele Einträge gibt es nach Genre?

In [ ]:
df.value_counts('genre')

Welche Interpret*innen kommen am häufigsten vor?

In [ ]:
df.interpret.value_counts()

Hier sehen wir schon, dass es insgesamt 91 Künstler*innen gibt. Das können wir uns aber auch direkt ausgeben lassen.

In [ ]:
len(df.interpret.unique())

In welchem Jahr ist der erste Song erschienen und wann der letzte?

In [ ]:
[df.release_year.min(), df.release_year.max()]

Anscheinend gibt es einen Fehler in dem Datensatz! Wir schauen nach, welche Jahre es alles gibt.

In [ ]:
df.release_year.sort_values().unique()

Die Jahre `20066` und `20014` sind anscheinend Tippfehler und wir ersetzen sie.

In [ ]:
df.replace({'release_year': {20066: 2006,  20014: 2014}}, inplace=True)

In [ ]:
[df.release_year.min(), df.release_year.max()]

Wir können uns auch alle numerischen Werte eines Dataframes beschreiben lassen.

In [ ]:
df.describe()

Kommen wir nun zur Visualisierung. Als Backend benutzen wir das Tool plotly. Standardmäßig wird matplotlib benutzt. 

In [ ]:
pd.options.plotting.backend = "plotly"

Zuerst plotten wir die Verteilung der Erscheinungsjahre.

In [ ]:
release_year_fig = df.release_year.value_counts().sort_index().plot(kind='bar', title='Release Year')

In [ ]:
release_year_fig.show()

Eine Möglichkeit das gruppiert über die Genres zu betrachten ist eine Pivottabelle.

In [ ]:
genre_pivot = df.pivot_table(index='release_year', columns='genre', aggfunc='size', fill_value=0)
genre_pivot.head(n=10)

In [ ]:
genre_release_year_fig = genre_pivot.plot(kind='bar', title='Genre by Release Year')
genre_release_year_fig.update_layout(barmode='group')
genre_release_year_fig.show()

Was sind die jüngsten Songs in unserem Datensatz?

In [ ]:
df.sort_values('release_year', ascending=False).iloc[0:10]

Was sind die ersten Rapsongs?

In [ ]:
df[df['genre'] == 'Rap'].sort_values('release_year').head(n=20)

Wir können unseren Datensatz auch filtern.

In [ ]:
df[(df.release_year >= 1995) & (df.release_year < 2005)].sort_values('release_year')

Wir füngen eine neue Spalte `decade` hinzu, die sich als Berechnung aus der Spalte `release_year` ergibt.

In [ ]:
import math
df['decade'] = df['release_year'].map(lambda x : int(math.trunc(x / 10) * 10))

Plotten wir die Verteilung der Einträge nochmal, dieses Mal nach Jahrzehnt.

In [ ]:
genre_pivot_decade = df.pivot_table(index='decade', columns='genre', aggfunc='size', fill_value=0)

In [ ]:
genre_decade_fig = genre_pivot_decade.plot(kind='bar', title='Genre by Release Year')
genre_decade_fig.update_layout(barmode='group')
genre_decade_fig.show()

Zuletzt speichern wir den Datensatz noch als csv Datei.

In [ ]:
df.to_csv('../data/external/GermanSongLyricsCorpus/all.csv', index=False)